In [67]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [68]:
## Read the pdfs from the folder

loader=PyPDFDirectoryLoader("./us_census")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [69]:
len(final_documents)

316

In [70]:
## Embedding Using HuggingFace 
hugging_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  #sentence-transformers/all-MiniLM-16-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
) 

In [71]:
import numpy as np
print(np.array(hugging_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(hugging_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568495e-02 -1.19099207e-02 -3.37892659e-02  2.94559393e-02
  5.19159622e-02  5.73839657e-02 -4.10017520e-02  2.74268128e-02
 -1.05128177e-01 -1.58056151e-02  7.94858783e-02  5.64318821e-02
 -1.31765129e-02 -3.41544114e-02  5.81603125e-03  4.72548008e-02
 -1.30746663e-02  3.12994490e-03 -3.44226174e-02  3.08406297e-02
 -4.09086086e-02  3.52737866e-02 -2.43761614e-02 -4.35831547e-02
  2.41503231e-02  1.31986542e-02 -4.84449882e-03  1.92347541e-02
 -5.43912724e-02 -1.42735019e-01  5.15527185e-03  2.93115675e-02
 -5.60810901e-02 -8.53537116e-03  3.14141251e-02  2.76736785e-02
 -2.06188262e-02  8.24231356e-02  4.15425450e-02  5.79655431e-02
 -3.71587388e-02  6.26158481e-03 -2.41389982e-02 -5.61795617e-03
 -2.51715314e-02  5.04967617e-03 -2.52801143e-02 -2.91944970e-03
 -8.24045856e-03 -5.69604635e-02  2.30822880e-02 -5.54217817e-03
  5.11555672e-02  6.09937608e-02  6.49766102e-02 -5.38514256e-02
  2.19109654e-02 -2.54194438e-02 -4.49223146e-02  4.22459058e-02
  4.75252122e-02  7.23216

In [72]:
## VectorStore creation
vectorstores=FAISS.from_documents(final_documents[:120],hugging_embeddings)

In [73]:
## Query using Similiarity search
query="What is Health Insurance Coverage?"
relevant_documents=vectorstores.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [74]:
retriever=vectorstores.as_retriever(search_type="similarity",search_kwargs={'k':3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000297949F67D0> search_kwargs={'k': 3}


In [117]:
import os 
from dotenv import load_dotenv
load_dotenv()


True

In [118]:
from langchain_community.llms import HuggingFaceHub
hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={'temperature':0.1,"max_length":500},
    huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_TOKEN')
)

query="What is the health insurance coverage?"
hf.invoke(query)

BadRequestError: (Request ID: k8TxbVi-22YJbLYzY6h9_)

Bad request:
Authorization header is correct, but the token seems invalid

In [ ]:
## hugging face model can be run locally through the huggingface pipeline class
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
# hf= HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-7B-v0.1",
#     task="text-generation",
#     pipeline_kwargs={"temperature":0,"max_new_tokens":300}
# )

# llm=hf
# llm.invoke(query)

In [97]:
## Prompt template

prompt_template='''
    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    {context}
    Question:{question}

    Helpful Answers:
    
'''

In [98]:
prompt=PromptTemplate(template=prompt_template,input_variables=['context','question'])

In [99]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt},
)

In [100]:
query="""DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"""

In [101]:
# Call the QA chain with our query.
result=retrievalQA.invoke({"query":query})
print(result['result'])

BadRequestError: (Request ID: iltwYjUrgTF6j0n0n4nsb)

Bad request:
Authorization header is correct, but the token seems invalid